In [1]:
!pip install -r requirements.txt

In [2]:
!mkdir data
#
! wget "https://arxiv.org/pdf/1810.04805.pdf" -O ./data/BERT_arxiv.pdf
! wget "https://arxiv.org/pdf/2005.11401" -O ./data/RAG_arxiv.pdf
! wget "https://arxiv.org/pdf/2310.11511" -O ./data/self_rag_arxiv.pdf
! wget "https://arxiv.org/pdf/2401.15884" -O ./data/crag_arxiv.pdf

mkdir: cannot create directory ‘data’: File exists
--2024-09-22 18:04:52--  https://arxiv.org/pdf/1810.04805.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/1810.04805 [following]
--2024-09-22 18:04:52--  http://arxiv.org/pdf/1810.04805
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775166 (757K) [application/pdf]
Saving to: ‘./data/BERT_arxiv.pdf’

./data/BERT_arxiv.p 100%[===================>] 757.00K  --.-KB/s    in 0.05s   

2024-09-22 18:04:52 (16.3 MB/s) - ‘./data/BERT_arxiv.pdf’ saved [775166/775166]

--2024-09-22 18:04:52--  https://arxiv.org/pdf/2005.11401
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... conne

In [3]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool,QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters,FilterCondition
from typing import List,Optional

In [4]:
import  nest_asyncio
nest_asyncio.apply()

In [5]:
documents = SimpleDirectoryReader(input_files = ['./data/self_rag_arxiv.pdf']).load_data()
print(len(documents))
print(f"Document Metadata: {documents[0].metadata}")

30
Document Metadata: {'page_label': '1', 'file_name': 'self_rag_arxiv.pdf', 'file_path': 'data/self_rag_arxiv.pdf', 'file_type': 'application/pdf', 'file_size': 1405127, 'creation_date': '2024-09-22', 'last_modified_date': '2023-10-19'}


In [6]:
splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=100)
nodes = splitter.get_nodes_from_documents(documents)
print(f"Length of nodes : {len(nodes)}")
print(f"get the content for node 0 :{nodes[0].get_content(metadata_mode='all')}")

Length of nodes : 43
get the content for node 0 :page_label: 1
file_name: self_rag_arxiv.pdf
file_path: data/self_rag_arxiv.pdf
file_type: application/pdf
file_size: 1405127
creation_date: 2024-09-22
last_modified_date: 2023-10-19

Preprint.
SELF-RAG: LEARNING TO RETRIEVE , GENERATE ,AND
CRITIQUE THROUGH SELF-REFLECTION
Akari Asai†, Zeqiu Wu†, Yizhong Wang†§, Avirup Sil‡, Hannaneh Hajishirzi†§
†University of Washington§Allen Institute for AI‡IBM Research AI
{akari,zeqiuwu,yizhongw,hannaneh }@cs.washington.edu ,avi@us.ibm.com
ABSTRACT
Despite their remarkable capabilities, large language models (LLMs) often produce
responses containing factual inaccuracies due to their sole reliance on the paramet-
ric knowledge they encapsulate. Retrieval-Augmented Generation (RAG), an ad
hoc approach that augments LMs with retrieval of relevant knowledge, decreases
such issues. However, indiscriminately retrieving and incorporating a fixed number
of retrieved passages, regardless of whether retrieval 

In [7]:
import chromadb
db = chromadb.PersistentClient(path="./chroma_db_mistral")
chroma_collection = db.get_or_create_collection("multidocument-agent")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [8]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
#
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
#
Settings.embed_model = embed_model
#
Settings.chunk_size = 1024

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [30]:
from llama_index.llms.mistralai import MistralAI
import os
os.environ["MISTRAL_API_KEY"] = 'QKoiAPxq1tuxMhU1QnPXNUFbA0lTqs28'
llm = MistralAI(model="mistral-large-latest")

name = "BERT_arxiv"
vector_index = VectorStoreIndex(nodes,storage_context=storage_context)
vector_index.storage_context.vector_store.persist(persist_path="/content/chroma_db")
#
# Define Vectorstore Autoretrieval tool
def vector_query(query:str,page_numbers:Optional[List[str]]=None)->str:
  '''
  perform vector search over index on
  query(str): query string needs to be embedded
  page_numbers(List[str]): list of page numbers to be retrieved,
                          leave blank if we want to perform a vector search over all pages
  '''
  page_numbers = page_numbers or []
  metadata_dict = [{"key":'page_label',"value":p} for p in page_numbers]
  #
  query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(metadata_dict, condition=FilterCondition.OR),
    llm=llm
)
  #
  response = query_engine.query(query)
  return response
#
#llamiondex FunctionTool wraps any python function we feed it
vector_query_tool = FunctionTool.from_defaults(name=f"vector_tool_{name}",
                                              fn=vector_query)
# Prepare Summary Tool
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",
                                                      se_async=True, llm = llm)
summary_query_tool = QueryEngineTool.from_defaults(name=f"summary_tool_{name}",
                                                    query_engine=summary_query_engine,
                                                  description=("Use ONLY IF you want to get a holistic summary of the documents."
                                              "DO NOT USE if you have specified questions over the documents."))

print(f"Using LLM: {llm}")

Using LLM: callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x78218a991e40> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x782197071b40> completion_to_prompt=<function default_completion_to_prompt at 0x782196f877f0> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None model='mistral-large-latest' temperature=0.1 max_tokens=512 timeout=120.0 max_retries=5 random_seed=None additional_kwargs={}


In [31]:
response = llm.predict_and_call([vector_query_tool],
                                "Summarize the content in page number 2",
                                verbose=True)

print(f"Using LLM: {llm}")

=== Calling Function ===
Calling function: vector_tool_BERT_arxiv with args: {"query": "Summarize the content in page number 2", "page_numbers": ["2"]}
=== Function Output ===
The content on page 2 discusses the effectiveness of using reflection tokens for enhancing overall performance and customizing models during testing, such as balancing citation predictions and completeness. It also introduces the concept of Retrieval-Augmented Generation (RAG), which enhances language models by incorporating retrieved text passages, leading to significant improvements in tasks that require extensive knowledge. Additionally, it mentions a recent work that involves instruction-tuning a language model with a fixed number of examples.
Using LLM: callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x78218a991e40> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x782197071b40> completion_to_prompt=<function default_completion_to_prompt at 0x782196f877f0> ou

In [42]:
def get_doc_tools(file_path:str,name:str)->str:
  '''
  get vector query and sumnmary query tools from a document
  '''
  #load documents
  documents = SimpleDirectoryReader(input_files = [file_path]).load_data()
  print(f"length of nodes")
  splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=100)
  nodes = splitter.get_nodes_from_documents(documents)
  print(f"Length of nodes : {len(nodes)}")
  #instantiate Vectorstore
  vector_index = VectorStoreIndex(nodes,storage_context=storage_context)
  vector_index.storage_context.vector_store.persist(persist_path="/content/chroma_db")
  #
  # Define Vectorstore Autoretrieval tool
  def vector_query(query:str,page_numbers:Optional[List[str]]=None)->str:
    '''
    perform vector search over index on
    query(str): query string needs to be embedded
    page_numbers(List[str]): list of page numbers to be retrieved,
                            leave blank if we want to perform a vector search over all pages
    '''
    page_numbers = page_numbers or []
    metadata_dict = [{"key":'page_label',"value":p} for p in page_numbers]
    #
    query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(metadata_dict, condition=FilterCondition.OR),
    llm=llm
)
    #
    response = query_engine.query(query)
    return response
  #
  #llamiondex FunctionTool wraps any python function we feed it
  vector_query_tool = FunctionTool.from_defaults(name=f"vector_tool_{name}",
                                                fn=vector_query)
  # Prepare Summary Tool
  summary_index = SummaryIndex(nodes)
  summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",
                                                       se_async=True, llm = llm)
  summary_query_tool = QueryEngineTool.from_defaults(name=f"summary_tool_{name}",
                                                     query_engine=summary_query_engine,
                                                    description=("Use ONLY IF you want to get a holistic summary of the documents."
                                                "DO NOT USE if you have specified questions over the documents."))
  return vector_query_tool,summary_query_tool

In [43]:
import os
root_path = "/content/data"
file_name = []
file_path = []
for files in os.listdir(root_path):
  if files.endswith(".pdf"): # Use 'files' instead of 'file'
    file_name.append(files.split(".")[0])
    file_path.append(os.path.join(root_path,files)) # Use 'files' instead of 'file'

print(file_name)
print(file_path)

['crag_arxiv', 'BERT_arxiv', 'self_rag_arxiv', 'RAG_arxiv']
['/content/data/crag_arxiv.pdf', '/content/data/BERT_arxiv.pdf', '/content/data/self_rag_arxiv.pdf', '/content/data/RAG_arxiv.pdf']


In [44]:
papers_to_tools_dict = {}
for name,filename in zip(file_name,file_path):
  vector_query_tool,summary_query_tool = get_doc_tools(filename,name)
  papers_to_tools_dict[name] = [vector_query_tool,summary_query_tool]

length of nodes
Length of nodes : 22
length of nodes
Length of nodes : 28
length of nodes
Length of nodes : 43
length of nodes
Length of nodes : 30


In [45]:
initial_tools = [t for f in file_name for t in papers_to_tools_dict[f]]
initial_tools

In [46]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex
#
obj_index = ObjectIndex.from_objects(initial_tools,index_cls=VectorStoreIndex)
#

In [47]:
obj_retriever = obj_index.as_retriever(similarity_top_k=2)
tools = obj_retriever.retrieve("compare and contrast the papers self rag and corrective rag")
#
print(tools[0].metadata)
print(tools[1].metadata)

ToolMetadata(description='Use ONLY IF you want to get a holistic summary of the documents.DO NOT USE if you have specified questions over the documents.', name='summary_tool_self_rag_arxiv', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
ToolMetadata(description='vector_tool_self_rag_arxiv(query: str, page_numbers: Optional[List[str]] = None) -> str\n\n    perform vector search over index on\n    query(str): query string needs to be embedded\n    page_numbers(List[str]): list of page numbers to be retrieved,\n                            leave blank if we want to perform a vector search over all pages\n    ', name='vector_tool_self_rag_arxiv', fn_schema=<class 'llama_index.core.tools.utils.vector_tool_self_rag_arxiv'>, return_direct=False)


In [48]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
#
agent_worker = FunctionCallingAgentWorker.from_tools(tool_retriever=obj_retriever,
                                                     llm=llm,
                                                     system_prompt="""You are an agent designed to answer queries over a set of given papers.
                                                     Please always use the tools provided to answer a question.Do not rely on prior knowledge.""",
                                                     verbose=True)
agent = AgentRunner(agent_worker)

In [49]:
response = agent.query("Compare and contrast self rag and crag.")
print(str(response))

Added user message to memory: Compare and contrast self rag and crag.
=== Calling Function ===
Calling function: vector_tool_self_rag_arxiv with args: {"query": "Compare and contrast self rag and crag."}
=== Function Output ===
Encountered error: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}
=== Calling Function ===
Calling function: vector_tool_self_rag_arxiv with args: {"query": "Compare and contrast self rag and crag."}
=== Function Output ===
Self-RAG is an advanced retrieval-augmented generation (RAG) approach that incorporates a critic model to determine whether to retrieve information and which retrieved document to use for generation. It is designed to decide which action to trigger based on the retrieved information. On the other hand, CRAG is a plug-and-play approach that can be integrated into other frameworks, including Self-RAG. When CRAG is inserted into the Self-RAG framework, it replaces the retrieved items with processed internal knowledge f

In [50]:
response = agent.query("Summarize the paper corrective RAG.")
print(str(response))


Added user message to memory: Summarize the paper corrective RAG.
=== Calling Function ===
Calling function: summary_tool_self_rag_arxiv with args: {"input": "corrective RAG"}
=== Function Output ===
The correct term is "Retrieval-Augmented Generation" (RAG).
=== LLM Response ===
The correct term is "Retrieval-Augmented Generation" (RAG).
The correct term is "Retrieval-Augmented Generation" (RAG).
